In [20]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset,TensorDataset

from tqdm import tqdm_notebook

# Model Structure

In [62]:
class MF(nn.Module):
    def __init__(self, dim, num_users, num_items):
        super(MF, self).__init__()
        self.user_embeddings = nn.Embedding(num_users+1, dim)
        self.item_embeddings = nn.Embedding(num_items+1, dim)
        self.linear  = nn.Linear(dim, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, user_id, item_id):
        user_embedding = self.user_embeddings(user_id)
        item_embedding = self.item_embeddings(item_id)
        
        cross_value = user_embedding * item_embedding
        return cross_value
    
class MLP(nn.Module):
    def __init__(self, dim, num_users, num_items, layer_sizes):
        super(MLP, self).__init__()
        self.user_embeddings = nn.Embedding(num_users+1, dim)
        self.item_embeddings = nn.Embedding(num_items+1, dim)
        
        self.linears = []
        prev_size = dim * 2
        for layer in layer_sizes:
            self.linears.append(nn.Linear(prev_size, layer))
            prev_size = layer
    
        self.relu = nn.ReLU()
    def forward(self, user_id, item_id):
        user_embedding = self.user_embeddings(user_id)
        item_embedding = self.item_embeddings(item_id)
        
        input_embedding = torch.cat((user_embedding, item_embedding), 1)
        for linear in self.linears:
            input_embedding = linear(input_embedding)
            input_embedding = self.relu(input_embedding)
        return input_embedding
    
class NCF(nn.Module):
    def __init__(self, num_users, num_items, MF_dim, MLP_dim, MLP_layers):
        super(NCF,self).__init__()
        self.mf = MF(MF_dim, num_users, num_items)
        self.mlp = MLP(MLP_dim, num_users, num_items, MLP_layers)
        
        self.linear = nn.Linear(MF_dim + MLP_layers[-1], 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, user_id, item_id):
        MF_features = self.mf(user_id, item_id)
        MLP_features = self.mlp(user_id, item_id)
        
        input_features = torch.cat((MF_features, MLP_features), 1)
        X = self.linear(input_features)
        return self.sigmoid(X)

# Data

In [4]:
movie_1m_ratings = pd.read_csv("Data/ml-1m/ratings.dat", sep="::", names=['user_id', 'item_id', 'rating', 'timestamp'], engine="python")
num_users = movie_1m_ratings.user_id.unique().shape[0]
num_movies = movie_1m_ratings.item_id.unique().shape[0]

In [72]:
max_user_id = max(movie_1m_ratings.user_id)
max_movieId = max(movie_1m_ratings.item_id)

In [5]:
sparsity = 1 - len(movie_1m_ratings) / (num_users * num_movies)
print("number of users: {}, number of movies: {}, sparsity: {}".format(num_users, num_movies, sparsity))
movie_1m_ratings.head(5)

number of users: 6040, number of movies: 3706, sparsity: 0.9553163743776871


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
valid_user = []
for i, satisfied in enumerate(movie_1m_ratings.groupby("user_id").size() > 20):
    if satisfied:
        valid_user.append(i+1)

In [7]:
valid_ratings = movie_1m_ratings[movie_1m_ratings.user_id.isin(valid_user)]
valid_ratings

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [16]:
user_choice_dict = {}
for _, rating in valid_ratings.iterrows():
    user, item = rating["user_id"], rating["item_id"]
    if user not in user_choice_dict:
        user_choice_dict[user] = []
    user_choice_dict[user].append(item)
    
def generate_train_test(user_choice_dict, train_ratio, negative_count):
    X_train, y_train, X_test, y_test = [], [], [], []
    for user in user_choice_dict:
        viewed_movies = user_choice_dict[user]
        negative_sample = np.random.choice([movie for movie in range(1, num_movies+1) if movie not in viewed_movies], negative_count)
        
        #Positive train test split
        train_size, test_size = int(len(viewed_movies) * train_ratio), len(viewed_movies) - int(len(viewed_movies) * train_ratio)
        for movie in viewed_movies[:train_size]:
            X_train.append([user, movie])
            y_train.append(1)
        for movie in viewed_movies[:test_size]:
            X_test.append([user, movie])
            y_test.append(1)
        
        #Negative train test split
        train_neg_size, test_neg_size = int(len(negative_sample) * train_ratio), len(negative_sample) - int(len(negative_sample) * train_ratio)
        for movie in negative_sample[:train_size]:
            X_train.append([user, movie])
            y_train.append(0)
        for movie in negative_sample[:test_size]:
            X_test.append([user, movie])
            y_test.append(0)
        
    return np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
    
train_X, train_y, test_X, test_y = generate_train_test(user_choice_dict, 0.9, 100)

In [51]:
train_X, train_y, test_X, test_y = torch.from_numpy(train_X), torch.from_numpy(train_y), torch.from_numpy(test_X), torch.from_numpy(test_y),
train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)

In [52]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [128]:
def accuarcy(y_pred, y_true):
    return ((y_pred > 0.5) == y_true).float().mean()

def train_model(model, train_loader, test_loader, max_epoch, lr):
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)
    epoch_acc = evaluate(model, test_loader)
    print("epoch {}, test acc: {}".format(0, epoch_acc))
    for epoch in range(max_epoch):
        epoch_loss = 0
        step = 0
        
        for batch, labels in tqdm_notebook(train_loader):
            y_pred = model(batch[:, 0], batch[:, 1])
            loss = loss_fn(y_pred, labels.float())
            epoch_loss += loss
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            step += 1
        epoch_acc = evaluate(model, test_loader)
        print("epoch {} loss: {}, test acc: {}".format(epoch+1, epoch_loss / step, epoch_acc))
        
    return model

def evaluate(model, test_loader):
    users, movies, prediction, ground_true = [], [], [], []
    accs = []
    with torch.no_grad():
        for batch, label in test_loader:
            y_pred = model(batch[:, 0], batch[:, 1])
            
            users.append(batch[:, 0])
            movies.append(batch[:, 1])
            prediction.append(y_pred)
            ground_true.append(label)
    users = torch.cat(users)
    movies = torch.cat(movies)
    prediction = torch.cat(prediction).squeeze()
    ground_true = torch.cat(ground_true)
    #print(prediction.shape)
    acc = accuarcy(prediction, ground_true)
    return acc

In [130]:
ncf = NCF(max_user_id, max_movieId, 64, 64, [32, 16])
ncf = train_model(ncf, train_loader, test_loader, 10, 0.001)

epoch 0, test acc: 0.4986320734024048



epoch 1 loss: 0.3584807217121124, test acc: 0.7572786211967468



epoch 2 loss: 0.2996557056903839, test acc: 0.7574368715286255



epoch 3 loss: 0.2904259264469147, test acc: 0.7621378302574158



epoch 4 loss: 0.2762939929962158, test acc: 0.7817840576171875


...


epoch 6 loss: 0.22091038525104523, test acc: 0.8432491421699524



epoch 7 loss: 0.1875162124633789, test acc: 0.8723279237747192



epoch 8 loss: 0.1571355015039444, test acc: 0.8996355533599854



epoch 9 loss: 0.13143780827522278, test acc: 0.9151830673217773



epoch 10 loss: 0.11054947972297668, test acc: 0.9335684776306152
